In [1]:
import numpy as np
import pandas as pd
import os
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from multiprocessing import Pool as ThreadPool
from datetime import datetime
import time
from sklearn.preprocessing import LabelEncoder, StandardScaler
from gensim.models import word2vec
from sklearn.utils import shuffle
from tqdm import tqdm

In [2]:
data_dir='/home/uniml/work/hw/dataset/'
temp_save_dir='./temp_data/'
age_train_file_path=data_dir+'age_train.csv'
age_test_file_path=data_dir+'age_test.csv'
app_info_file_path=data_dir+'app_info.csv'
user_app_actived_file_path=data_dir+'user_app_actived.csv'
user_basic_info_file_path=data_dir+'user_basic_info.csv'
user_behavior_info_file_path=data_dir+'user_behavior_info.csv'
user_app_usage_file_path=data_dir+'user_app_usage.csv'

In [3]:
embedding_size=128

In [4]:
app_actived_info=pd.read_csv(user_app_actived_file_path,header=None,names=['uId','app_info'])

In [5]:
app_actived_list=np.array(app_actived_info['app_info'].apply(lambda x:shuffle(x.strip().split('#'))))

In [6]:
print('te')

te


In [7]:
model = word2vec.Word2Vec(app_actived_list, size=embedding_size,window=5,iter=20,sg=1,workers=25,compute_loss=True)

In [8]:
model.save('./app_info_word2vec_windows_5_size_128_iter_20_1.model')

/opt/conda/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
result={
    'avg':[],
    'sum':[]
}
for i in tqdm(app_actived_list):
    no_exist_count=0
    try:
        user_actived_info=np.array(model[i]).sum(aixs=1)
        result['sum'].append(user_actived_info)
        result['avg'].append(user_actived_info/(len(i)-no_exist_count))
    except:
        user_actived_info=np.array([0 for _ in range(embedding_size)],dtype=np.float32)
        for app_id in i:
            if model.wv.__contains__(app_id):
                user_actived_info+=model[app_id]
            else:
                no_exist_count+=1
        result['sum'].append(user_actived_info)
        if len(i)-no_exist_count==0:
            result['avg'].append(user_actived_info)
        else:
            result['avg'].append(user_actived_info/(len(i)-no_exist_count))

  0%|          | 0/4999341 [00:00<?, ?it/s]/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app
100%|██████████| 4999341/4999341 [23:56<00:00, 3480.46it/s]


In [10]:
print(len(result['avg']),len(result['sum']))

4999341 4999341


In [11]:
result_avg=pd.DataFrame(result['avg'],dtype=np.float32)
result_avg.rename(columns=lambda x:str(x)+'_avg',inplace=True)
result_avg_with_uid=pd.concat([app_actived_info,result_avg],axis=1)
_=result_avg_with_uid.pop('app_info')
h5 = pd.HDFStore(temp_save_dir+'app_actived_embedding_by_w2v_avg_windows_5_size_128_iter_20_1.h5','w',complevel=4,complib='blosc')
h5['data'] = result_avg_with_uid
h5.close()

In [12]:
result_sum=pd.DataFrame(result['sum'],dtype=np.float32)
result_sum.rename(columns=lambda x:str(x)+'_sum',inplace=True)
result_sum_with_uid=pd.concat([app_actived_info,result_sum],axis=1)
_=result_sum_with_uid.pop('app_info')
h5 = pd.HDFStore(temp_save_dir+'app_actived_embedding_by_w2v_sum_windows_5_size_128_iter_20_1.h5','w',complevel=4,complib='blosc')
h5['data'] = result_sum_with_uid
h5.close()

In [ ]:
app_usage_info=pd.read_hdf(data_dir+'user_app_usage_df.h5',key='data')

In [ ]:
app_usage_list=np.array(app_usage_info['appIds'].apply(lambda x:shuffle(str(x).strip().split('#'))))

In [ ]:
model_usage = word2vec.Word2Vec(app_usage_list, size=embedding_size,window=5,iter=20,sg=1, workers=25,compute_loss=True)

In [ ]:
model_usage.save('./app_usage_word2vec_windows_5_size_128_iter_20.model')

In [ ]:
result={
    'avg':[],
    'sum':[]
}
for i in app_usage_list:
    no_exist_count=0
    try:
        user_actived_info=np.array(model_usage[i]).sum(aixs=1)
        result['sum'].append(user_actived_info)
        result['avg'].append(user_actived_info/(len(i)-no_exist_count))
    except:
        user_actived_info=np.array([0 for _ in range(embedding_size)],dtype=np.float64)
        for app_id in i:
            if model_usage.wv.__contains__(app_id):
                user_actived_info+=model_usage[app_id]
            else:
                no_exist_count+=1
        result['sum'].append(user_actived_info)
        if len(i)-no_exist_count==0:
            result['avg'].append(user_actived_info)
        else:
            result['avg'].append(user_actived_info/(len(i)-no_exist_count))

In [ ]:
result_avg=pd.DataFrame(result['avg'],dtype=np.float32)
result_avg.rename(columns=lambda x:str(x)+'_avg',inplace=True)
result_avg_with_uid=pd.concat([app_usage_info,result_avg],axis=1)
_=result_avg_with_uid.pop('appIds')
h5 = pd.HDFStore(temp_save_dir+'app_usage_embedding_by_w2v_avg_windows_5_size_128_iter_20_1.h5','w',complevel=4,complib='blosc')
h5['data'] = result_avg_with_uid
h5.close()

In [ ]:
result_sum=pd.DataFrame(result['sum'],dtype=np.float32)
result_sum.rename(columns=lambda x:str(x)+'_sum',inplace=True)
result_sum_with_uid=pd.concat([app_usage_info,result_sum],axis=1)
_=result_sum_with_uid.pop('appIds')
h5 = pd.HDFStore(temp_save_dir+'app_usage_embedding_by_w2v_sum_windows_5_size_128_iter_20_1.h5','w',complevel=4,complib='blosc')
h5['data'] = result_sum_with_uid
h5.close()